In [16]:
import sys
# !{sys.executable} -m pip install imblearn

In [17]:
import sys
#!{sys.executable} -m pip install xgboost

In [26]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.cross_validation import KFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score
from sklearn.pipeline import Pipeline
from xgboost.sklearn import XGBClassifier

from datetime import datetime
from datetime import timedelta

In [27]:
train = pd.read_csv("./all/train.csv")
weather = pd.read_csv("./all/weather.csv")
spray = pd.read_csv("./all/spray.csv")

In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
Date                      10506 non-null object
Address                   10506 non-null object
Species                   10506 non-null object
Block                     10506 non-null int64
Street                    10506 non-null object
Trap                      10506 non-null object
AddressNumberAndStreet    10506 non-null object
Latitude                  10506 non-null float64
Longitude                 10506 non-null float64
AddressAccuracy           10506 non-null int64
NumMosquitos              10506 non-null int64
WnvPresent                10506 non-null int64
dtypes: float64(2), int64(4), object(6)
memory usage: 985.0+ KB


In [29]:
spray.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 4 columns):
Date         14835 non-null object
Time         14251 non-null object
Latitude     14835 non-null float64
Longitude    14835 non-null float64
dtypes: float64(2), object(2)
memory usage: 463.7+ KB


In [30]:
train.shape, weather.shape, spray.shape

((10506, 12), (2944, 22), (14835, 4))

In [31]:
train['WnvPresent'].value_counts()

0    9955
1     551
Name: WnvPresent, dtype: int64

In [32]:
551 / (551 + 9955)

0.05244622120692937

In [33]:
train.Date = pd.to_datetime(df.Date)
weather.Date = pd.to_datetime(weather.Date)
spray.Date = pd.to_datetime(spray.Date)

NameError: name 'df' is not defined

In [ ]:
mapdata = np.loadtxt("./all/mapdata_copyright_openstreetmap_contributors.txt")
traps = pd.read_csv('./all/train.csv')[['Date', 'Trap','Longitude', 'Latitude', 'WnvPresent']]

aspect = mapdata.shape[0] * 1.0 / mapdata.shape[1]
lon_lat_box = (-88, -87.5, 41.6, 42.1)

#plot map
plt.figure(figsize=(10,14))
plt.imshow(mapdata, cmap=plt.get_cmap('gray'), extent=lon_lat_box, aspect=aspect)

#Spray locations
sprays = spray[['Longitude', 'Latitude']].drop_duplicates()
sprays = sprays[sprays['Latitude'] < 42.3]  #outliers excluded
plt.scatter(sprays['Longitude'], sprays['Latitude'], marker='*', color='orange',alpha=.3, label='Spray Locations')

#Trap locations
locations = traps[['Longitude', 'Latitude']].drop_duplicates().values
plt.scatter(locations[:,0], locations[:,1], marker='x', label='Trap Locations')

#Weather locations
plt.scatter(x = (-87.933, -87.752), y = (41.995, 41.786), marker='o', color='r', label='Weather Station')
            
plt.title('West Nile Virus Preventions in Chicago')
plt.legend(frameon=1)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.savefig('heatmap.png');

In [ ]:
#check columns for the value with M
object_features = ['Tavg', 'Depart', 'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset',
       'CodeSum', 'Depth', 'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure',
       'SeaLevel', 'AvgSpeed']
for col in weather[object_features]:
    station_1 = len(weather[(weather[col].str.contains('M')) & (weather.Station==1)])
    print(col + ' has ' + str(station_1) + ' missing values at station 1')
    station_2 = len(weather[(weather[col].str.contains('M')) & (weather.Station==2)])
    print(col + ' has ' + str(station_2) + ' missing values at station 2')

In [ ]:
#check columns for values with T
for col in weather[object_features]:
    station_1 = len(weather[(weather[col].str.contains('T')) & (weather.Station==1)])
    print(col + ' has ' + str(station_1) + ' missing values at station 1')
    station_2 = len(weather[(weather[col].str.contains('T')) & (weather.Station==2)])
    print(col + ' has ' + str(station_2) + ' missing values at station 2')

In [ ]:
weather.info()

In [ ]:
weather.select_dtypes(include='object').columns

In [ ]:
#columns to replace values of M, T, - 
columns = ['Tavg', 'Depart', 'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset',
           'SnowFall', 'PrecipTotal', 'StnPressure','SeaLevel', 'AvgSpeed']

for column in columns:
    weather.replace({'M': None}, inplace = True)
    weather.replace({'T': '0.00001'}, inplace = True)
    weather.replace({'  T': '0.00001'}, inplace = True)
    weather.replace({'-': None}, inplace = True)

In [ ]:
# I didn't find these columns useful at all
weather.drop(columns= ['CodeSum', 'Depth', 'Water1'],inplace=True)

In [ ]:
#convert columns to float type and fill na with mean
for column in columns:
    weather[column] = weather[column].astype(float)
    weather[column] = weather[column].fillna(weather[column].mean(skipna = True))

In [ ]:
#create new dataframe with the average of columns for the two weather stations
weather_both = pd.DataFrame()
station_1 = weather[weather['Station'] == 1].reset_index()
station_2 = weather[weather['Station'] == 2].reset_index()
weather_both['Date'] = station_1['Date']


def avg_station(df):
    for col in df:
        weather_both[col] = (station_1[col] + station_2[col])*.5

In [ ]:
#apply function
avg_station(weather.drop(['Date','Station'],axis=1))

In [ ]:
#create a column for the days since it last rained
weather_both["days_since_rain"] = 0
days = 0

for i in range(weather_both.shape[0]):
    if weather_both.loc[i,'PrecipTotal'] == 0:
        days = days + 1
        weather_both.loc[i,'days_since_rain'] = days
    else:
        weather_both.loc[i, 'days_since_rain'] = 0
        days=0

In [ ]:
#check to make sure everything is good so far
weather_both.head(2)

In [ ]:
#set a new dataframe for the features i want to do a time lag on
var = ['Tmax', 'Tmin', 'Tavg', 'DewPoint', 'WetBulb', 'SnowFall',
       'PrecipTotal', 'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed']

lag_features = weather_both[var]

In [ ]:
#set the number of lags i want, these are the lags in days
lags = (1,3,7,14)

#set to a final dataframe
#.assign assigns new columns to the dataframe, after that is a list comprehension
#f'{col}_lag_{n}' <-- f' string feature to assign column name, its like .format()
#list comp <-- for every column shift down for every lag
final_weather = weather_both.assign(**{f'{col}_lag_{n}': 
                                       lag_features[col].shift(n) for n in lags for col in lag_features})

In [ ]:
#merges the two datasets together (train and final weather data)
result = train.merge(final_weather, on='Date')

In [ ]:
#function for how long a day is in minutes
def day_length(row):
    sunset = (round(row.Sunset / 100) * 60)
    sunrise = (round(row.Sunrise / 100) * 60)
    return int(abs(sunset - sunrise))

In [ ]:
#apply the function
result['day_length'] = result.apply(day_length, axis=1)

In [ ]:
result.select_dtypes(include='object').columns

In [ ]:
result.select_dtypes(include=['float64', 'int64']).columns

In [ ]:
columns_high_corr = abs(result.corr()["WnvPresent"]).sort_values(ascending=False).round(3).index[2:40]

In [ ]:
result.shape

In [ ]:
result["WnvPresent"].value_counts()

Select the five best features

In [ ]:
#training_features = result.drop(columns=['WnvPresent', 'NumMosquitos'], axis = 1)
training_features = result[columns_high_corr]
training_target = result["WnvPresent"]

In [ ]:
training_features.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(training_features, training_target,
                                                  test_size = .2,
                                                  random_state=11, shuffle = True)

Oversampling 

In [ ]:
sm = SMOTE(random_state=12, ratio = 1.0)
x_train_mod, y_train_mod = sm.fit_sample(x_train, y_train)
x_test_mod, y_test_mod = sm.fit_sample(x_test, y_test)

Logistic Regression

In [ ]:
lg = LogisticRegression()

print("Cross validation on over sampled train data:\n",cross_val_score(lg, x_train_mod,y_train_mod,n_jobs=-1, cv = 10).round(3),"\n")

print("Cross validation on test data: \n", cross_val_score(lg, x_test,y_test,n_jobs=-1, cv = 10).round(3), "\n")

In [ ]:
lg.fit(x_train_mod, y_train_mod)
predicted = lg.predict(x_train_mod)
print("R2 score on train dataset: ", lg.score(x_train_mod, y_train_mod).round(3))
print("ROC score on train dataset: ", roc_auc_score(y_train_mod, predicted).round(3))

In [ ]:
print("R2 score on test dataset: ", lg.score(x_test, y_test).round(3))
predicted = lg.predict(x_test)
print("ROC score on test dataset: ",roc_auc_score(y_test, predicted).round(3))

Gradient Boosting Classifier

In [ ]:
gbc = GradientBoostingClassifier()

In [ ]:
gbc = GradientBoostingClassifier()
print("Cross validation on train dataset: \n", cross_val_score(gbc, x_train_mod,y_train_mod, n_jobs=-1, cv = 10).round(3), "\n")
print("Cross validation on test dataset: \n", cross_val_score(gbc, x_test,y_test, n_jobs=-1, cv = 10).round(3), "\n")

In [ ]:
gbc.fit(x_train_mod, y_train_mod)
predicted = gbc.predict(x_train_mod)
print("R2 score on train dataset: ", gbc.score(x_train_mod, y_train_mod).round(3))
print("ROC score on train dataset: ", roc_auc_score(y_train_mod ,predicted).round(3))

In [ ]:
predicted = gbc.predict(x_test)
print("R2 score on test dataset: ", gbc.score(x_test, y_test).round(3))
print("ROC score on test dataset: ", roc_auc_score(y_test, predicted).round(3))

In [ ]:
GradientBoostingClassifier()

In [ ]:
# Using gridsearch CV to optimize results
score_function = make_scorer(roc_auc_score)
params_gbc = {
    "gbc__learning_rate"  : [1],
    "gbc__n_estimators"     : [10, 100, 300],
    "gbc__max_depth"  : [3, 10],
    "gbc__random_state"    : [42]
             }
steps_gbc = [('gbc', GradientBoostingClassifier())]
pipe_gbc = Pipeline(steps = steps_gbc)
gs_gbc = GridSearchCV(pipe_gbc, param_grid = params_gbc , scoring = score_function, verbose=1)

In [ ]:
gs_gbc.fit(x_train_mod, y_train_mod)

In [ ]:
gs_gbc.best_params_

In [ ]:
# Test gbc using gridserachcv updated results
gbc = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, random_state=42, max_depth=10)
gbc.fit(x_train_mod, y_train_mod)
predicted = gbc.predict(x_train_mod)
print("R2 score on train dataset: ", gbc.score(x_train_mod, y_train_mod).round(3))
print("ROC score on train dataset: ", roc_auc_score(y_train_mod ,predicted).round(3))

In [ ]:
predicted = gbc.predict(x_test)
print("R2 score on test dataset: ", gbc.score(x_test, y_test).round(3))
print("ROC score on test dataset: ", roc_auc_score(y_test, predicted).round(3))

In [ ]:
# Fine tune on accuracy score:
# {'gbc__learning_rate': 0.1,
# 'gbc__max_depth': 10,
# 'gbc__n_estimators': 500,
# 'gbc__random_state': 42}
# R2 score on train dataset:  0.903
# ROC score on train dataset:  0.903
# R2 score on test dataset:  0.846
# ROC score on test dataset:  0.682   too bad on ROC score, switch to ROC score for score function

In [ ]:
# gbc = GradientBoostingClassifier(learning_rate=1, n_estimators=100, random_state=42, max_depth=10)
# R2 score on train dataset:  0.962
# ROC score on train dataset:  0.962
# R2 score on test dataset:  0.903
# ROC score on test dataset:  0.625

In [ ]:
# gbc = GradientBoostingClassifier(learning_rate=0.1, n_estimators=500, random_state=42, max_depth=10)
# R2 score on train dataset:  0.965
# ROC score on train dataset:  0.965
# R2 score on test dataset:  0.92
# ROC score on test dataset:  0.607

Random Forest Classifier

In [14]:
rfc = RandomForestClassifier(max_depth=5)
#print("Cross validation score on train dataset: ", cross_val_score(rfc, x_train_mod,y_train_mod, n_jobs=-1).round(3), "\n")
#print("Cross validation score on test dataset: ", cross_val_score(rfc, x_test,y_test, n_jobs=-1).round(3), "\n")

In [15]:
rfc.fit(x_train_mod, y_train_mod)
print("R2 score on train dataset: ", rfc.score(x_train_mod, y_train_mod).round(3))
predicted = rfc.predict(x_train)
print("ROC score on train dataset: ", roc_auc_score(y_train, predicted).round(3))

NameError: name 'x_train_mod' is not defined

In [ ]:
print("R2 score on test dataset: ", rfc.score(x_test, y_test).round(3))
predicted = rfc.predict(x_test)
print("ROC score on test dataset: ", roc_auc_score(y_test, predicted).round(3))

** XGBOOST Model **

In [661]:
xgb = XGBClassifier(max_depth=7,
                           min_child_weight=1,
                           learning_rate=0.1,
                           n_estimators=500,
                           silent=True,
                           objective='binary:logistic',
                           gamma=0,
                           max_delta_step=0,
                           subsample=1,
                           colsample_bytree=1,
                           colsample_bylevel=1,
                           reg_alpha=0,
                           reg_lambda=0,
                           scale_pos_weight=1,
                           seed=1,
                           missing=None)
xgb.fit(x_train_mod, y_train_mod, eval_metric='auc', verbose=False, early_stopping_rounds=100)
y_pre = xgb.predict(x_test)
y_pro = xgb.predict_proba(x_test)[:, 1]


IndexError: list index out of range

** Create Sumission File for Kaggle **

In [570]:
test_kaggle = pd.read_csv("./all/test.csv")

In [571]:
test_kaggle.shape

(116293, 11)

In [572]:
test_kaggle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116293 entries, 0 to 116292
Data columns (total 11 columns):
Id                        116293 non-null int64
Date                      116293 non-null object
Address                   116293 non-null object
Species                   116293 non-null object
Block                     116293 non-null int64
Street                    116293 non-null object
Trap                      116293 non-null object
AddressNumberAndStreet    116293 non-null object
Latitude                  116293 non-null float64
Longitude                 116293 non-null float64
AddressAccuracy           116293 non-null int64
dtypes: float64(2), int64(3), object(6)
memory usage: 9.8+ MB


In [573]:
# We need to apply the same feature engineering steps for better prediction

In [574]:
test_kaggle.Date = pd.to_datetime(test_kaggle.Date)

In [575]:
columns_high_corr

Index(['day_length', 'DewPoint_lag_1', 'Sunrise', 'WetBulb_lag_1',
       'AvgSpeed_lag_14', 'Tmin_lag_1', 'DewPoint_lag_7', 'DewPoint',
       'Tavg_lag_14', 'ResultSpeed_lag_7', 'Tmin_lag_14', 'WetBulb_lag_7',
       'WetBulb', 'ResultSpeed_lag_14', 'Tmin_lag_7', 'WetBulb_lag_14',
       'Tavg_lag_1', 'SeaLevel_lag_3', 'Tmin', 'Tmax_lag_14', 'Sunset',
       'Tavg_lag_7', 'Tavg', 'DewPoint_lag_14', 'Longitude', 'AvgSpeed_lag_7',
       'Cool', 'Depart', 'Heat', 'DewPoint_lag_3', 'Tmax_lag_1',
       'PrecipTotal_lag_1', 'Tmax', 'ResultSpeed', 'ResultSpeed_lag_1',
       'ResultDir_lag_1', 'Tmax_lag_7', 'PrecipTotal_lag_3'],
      dtype='object')

In [576]:
test_feat_eng = test_kaggle.merge(final_weather, on='Date')
test_feat_eng['day_length'] = test_feat_eng.apply(day_length, axis=1)

In [577]:
test_feat_eng.shape

(116293, 74)

In [578]:
columns_high_corr

Index(['day_length', 'DewPoint_lag_1', 'Sunrise', 'WetBulb_lag_1',
       'AvgSpeed_lag_14', 'Tmin_lag_1', 'DewPoint_lag_7', 'DewPoint',
       'Tavg_lag_14', 'ResultSpeed_lag_7', 'Tmin_lag_14', 'WetBulb_lag_7',
       'WetBulb', 'ResultSpeed_lag_14', 'Tmin_lag_7', 'WetBulb_lag_14',
       'Tavg_lag_1', 'SeaLevel_lag_3', 'Tmin', 'Tmax_lag_14', 'Sunset',
       'Tavg_lag_7', 'Tavg', 'DewPoint_lag_14', 'Longitude', 'AvgSpeed_lag_7',
       'Cool', 'Depart', 'Heat', 'DewPoint_lag_3', 'Tmax_lag_1',
       'PrecipTotal_lag_1', 'Tmax', 'ResultSpeed', 'ResultSpeed_lag_1',
       'ResultDir_lag_1', 'Tmax_lag_7', 'PrecipTotal_lag_3'],
      dtype='object')

In [579]:
test_feat_eng = test_feat_eng[columns_high_corr]
test_feat_eng.shape

(116293, 38)

In [580]:
test_feat_eng.shape

(116293, 38)

In [650]:
test_kaggle_pred = gbc.predict_proba(test_feat_eng)
test_kaggle_pred[0:5]

array([[0.98309036, 0.01690964],
       [0.98309036, 0.01690964],
       [0.98309036, 0.01690964],
       [0.98309036, 0.01690964],
       [0.98309036, 0.01690964]])

** GBC submission **

In [651]:
submission = pd.DataFrame(gbc.predict_proba(test_feat_eng)[:,1], columns = ['WnvPresent'])

In [652]:
submission.set_index(np.arange(1, test_feat_eng.shape[0] + 1), inplace=True)
submission = submission.reset_index().rename(columns = {'index':'Id'})
submission.to_csv('./submission_gbc_default.csv', index = False)

In [653]:
pd_test_pred.value_counts()

0    82117
1    34176
dtype: int64

In [585]:
# linear regression
44504 / (71789 + 44504)

0.38268855391124146

In [586]:
71789 + 44504

116293